# Prepping Data for Classifying 
_Alicia Roberts_

Given data from the Hoji Shinbun Digital Archive for occurences of nisei in texts, clean the data to be ready for classifying. 

_goal:_ create a tokenized, shortened, cleaned string of each occurence of nisei in the text as its own data point in a data frame. 

input columns: link, linkWithDate, numWords, paragraphs, pdfName

Given num words, this will add N-1 data points per a data point where N is the value in numWords. 

outputs: 

training matrix: training values for each data point (leave all information up to some lower limit, then perform PCA later)
data matrix: link, year, text (not tokenized) 



Date started: 12/6/2024

In [34]:
# working with the output file Ryan made:

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style()


In [35]:
data = pd.read_json('output.json')

In [36]:
data.head()

,link,linkWithDate,numWords,paragraphs,pdfName
0,https://hojishinbun.hoover.org/?a=d&d=jan19360...,https://hojishinbun.hoover.org/en/newspapers/j...,5,滅念祀迎獸轚鼴罾鐮 81 帝 1 世を黏レゎそれ —№ 11 にこ # は於 & しヤやニル...,jan_19360710_0019.pdf
1,https://hojishinbun.hoover.org/?a=d&d=kam19350...,https://hojishinbun.hoover.org/en/newspapers/k...,7,"「 ，！ 一 ： こ匿 - — 511. :.?-<-€¢€1—""--#118*141( -...",kam_19350811_0003.pdf
2,https://hojishinbun.hoover.org/?a=d&d=kam19420...,https://hojishinbun.hoover.org/en/newspapers/k...,5,一世二世の失業者は ’： 一 1^ ノ至急市協へ報吿せょ一 ：1 - ”£2” 日本人救濟の...,kam_19420128_0003.pdf
3,https://hojishinbun.hoover.org/?a=d&d=jan19410...,https://hojishinbun.hoover.org/en/newspapers/j...,6,號ニ十六百七千四万一第 當太和 3 声では開菜一一一却年 2.. ' と I て來る四 ：：...,jan_19410107_0005.pdf
4,https://hojishinbun.hoover.org/?a=d&d=kam19350...,https://hojishinbun.hoover.org/en/newspapers/k...,2,ジブラルタルとスュズ運河 ， 藤丼整 霉とな 〇 先が ^ ブチルタル線杯か 6111 8 ...,kam_19350922_0002.pdf


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   link          100 non-null    object
 1   linkWithDate  100 non-null    object
 2   numWords      100 non-null    int64 
 3   paragraphs    100 non-null    object
 4   pdfName       100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [41]:
# testing the format of the texts:

type(data.iloc[0]['paragraphs']) # okay cool its the type we want! Let's run the cleaning on it?

str

In [42]:
def remove_hit(string, hit):
    '''return a shortened version of STRING that is centered around HIT with N characters on each side of it
    STRING: any string
    HIT: any word
    returns:
    STRING wihout HIT, N: any positive integer that is the location of HIT in STRING. will return the first occurence of the first character of HIT'''
    ns = [] # this means there is no occurence of HIT if empty
    mod_string = ''
    
    if (hit in string): # first see that HIT is actually in STRING to avoid errors
        size_hit = len(hit) # how many characters to examine at once 
        
        for n in range(len(string) - size_hit): # itterate through STRING until you reach HIT
            
            if string[n:n+size_hit] == hit: # iterating till we reach HIT
                if mod_string == '':
                    mod_string = string[0:n] + string[n+size_hit:] # create a modified string without HIT
                    
                else:
                    mod_string = mod_string[0:n - size_hit] + mod_string[n:] # since mod_string is already 3 indeces shorter, you have to account for that 
                ns.append(n)    
        if len(ns) == 1:
            return np.array(mod_string), ns[0]# return modified string + index value of the first occurence 
        return np.array(mod_string), ns
    
    # if HIT is not in STRING, return empty string and -1 (to be removed later):
    return '', -1 #

In [44]:
remove_hit(data.iloc[0]['paragraphs'], '二世')

(array('滅念祀迎獸轚鼴罾鐮 81 帝 1 世を黏レゎそれ —№ 11 にこ # は於 & しヤやニル ^ 」 81X2 する怒 5 ざ 511^ 牝把の 1251 にあ 61 蕾 ！！ 膽一 ^ 十 111 萬ミ & せられるが 、 この内 ，1 世は六萬 ， は七錢 2 ぃ体 “， しか . 〇 11& のや 5^ 叱は 71.-1- 六纪 5 傭へら IX てゐ 6 嫌こ 1 ヒパ 4 才 <1 ャ — 1 し \' I - 歡祀 41 はらつて ！ ヒげた蟓譬たる二说に ！！^ ！ す翠 . -5 づぃに - を初 13 つて 611 のであ 6 ゲ後 1 世 5 歡は撕少 08 をた ?\' る一 ^ V あつてそれに比して ， 二 #， 芑坩 11 奶 5 七此處十印以内に & は完ゲに 6 世の屮 2 なつて來るだらぅ 0 1 世パ 4 ォ爲ャ — 0) 譏 ！^ を籲保 4|) たて — 行かなければならなぃ 1 一世の資任や纗め X 纛 4^ ある “ A この 1^ 阏 11 社 # にあつてこ赏の ？？^ 糴濟 2 し \\3！ も & されてゐ 6 のは (11 山 ^58^ ふ — グメントで #) る ，8：|^ #18 も二 V の練 5 屮 ？ じて ” 广 58 を 3 へて I " 袭布 2 酽駔昏淀にデダ 1131^ 命 ：# #?\' 鳩に績布讓曹 1 立を )¢, 6!: いた 〇 た ， 北は泠濬 ， 南は我く 9 サ齡デ — ザ . 帝 N1 牛蒙 . 軍は |1|1 中離 |： ゎたって 011 する & 4-(: 遍ん艺すも 111 ££2 か ^^^ 氍 .1& 纪して ^ 年ご 2 1: 令装允勢 .^ 11 す .4 个打の匯令上 .0^8^83^ 二世 3 2 21 に ！！ 者粑供邦人 №1- ¢)?? 拿な ！ 3 れ 2 なつてゐ 6 が ， これが 55 令幕ねてみた ！^. 路 £££6 1^ 饞は實に 111 45 し I す - まず臌が 0： 處 <1. = 年来の 115, 感允想し得たであらぅか ， が盱繡 3 な办 ， 令 # 的なものまなつたの ！^^ 六 ， 七厶〆か 1 が爲 #|| したの II 一九一三的であつ卜 ： 费キの二 2 年には今 11 0) 争素 II 來 ^

In [45]:
data.iloc[0]['paragraphs']

'滅念祀迎獸轚鼴罾鐮 81 帝 1 世を黏レゎそれ —№ 11 にこ # は於 & しヤやニル ^ 」 81X2 する怒 5 ざ 511^ 牝把の 1251 にあ 61 蕾 ！！ 膽一 ^ 十 111 萬ミ & せられるが 、 この内 ，1 世は六萬 ， 二世は七錢 2 ぃ体 “， しか . 〇 11& のや 5^ 叱は 71.-1- 六纪 5 傭へら IX てゐ 6 嫌こ 1 ヒパ 4 才 <1 ャ — 1 し \' I - 歡祀 41 はらつて ！ ヒげた蟓譬たる二说に ！！^ ！ す翠 . -5 づぃに - を初 13 つて 611 のであ 6 ゲ後 1 世 5 歡は撕少 08 をた ?\' る一 ^ V あつてそれに比して ， 二 #， 芑坩 11 奶 5 七此處十印以内に & は完ゲに二世 6 世の屮 2 なつて來るだらぅ 0 1 世パ 4 ォ爲ャ — 0) 譏 ！^ を籲保 4|) たて — 行かなければならなぃ 1 一世の資任や纗め X 纛 4^ ある “ A この 1^ 阏 11 社 # にあつてこ赏の ？？^ 糴濟 2 し \\3！ も & されてゐ 6 のは (11 山 ^58^ ふ — グメントで #) る ，8：|^ #18 も二 V の練 5 屮 ？ じて ” 广 58 を 3 へて I " 袭布 2 酽駔昏淀にデダ 1131^ 命 ：# #?\' 鳩に績布讓曹 1 立を )¢, 6!: いた 〇 た ， 北は泠濬 ， 南は我く 9 サ齡デ — ザ . 帝 N1 牛蒙 . 軍は |1|1 中離 |： ゎたって 011 する & 4-(: 遍ん艺すも 111 ££2 か ^^^ 氍 .1& 纪して ^ 年ご 2 1: 令装允勢 .^ 11 す .4 个打の匯令上 .0^8^83^ 二世の和 3 2 21 に ！！ 者粑供邦人 №1- ¢)?? 拿な ！ 3 れ 2 なつてゐ 6 が ， これが 55 令幕ねてみた ！^. 路 £££6 1^ 饞は實に 111 45 し I す - まず臌が 0： 處 <1. = 年来の 115, 感允想し得たであらぅか ， が盱繡 3 な办 ， 令 # 的なものまなつたの ！^^ 六 ， 七厶〆か 1 が爲 #|| したの II 一九一三的であつ卜 ： 费キの二 2 年には今 11 0) 争素 II 來 ^~